# DAT312 - Developing and deploying a SAP HANA ML scenario

### Introduction - Telco Customer Churn
- This scenario is based on telco customer data and focuses on predicting which customers are likely to churn, i.e. likely to cancel their contract next month.
- Given is a data set CUSTOMERCHURN, which provides insight about each telco customer based on standard customer attributes and measures on current, previous month (_PM) and previous-previous month (_PPM) statistics. 
- This data set comprises historic information, hence the churn behavior is known (ContractActivityLabel) from the past, and is thus used for training a classification model. 
- A second data set NEWCUSTOMERCHURN comprises data from a new or the current month, describing the current customer state, where the churn status is not known and shall be predicted in order to take actions targeting customers which are likely to churn (cancel their contract and leave).


### Exercise 1 - Build a ML scenario using the Python API for HANA ML
- Build a simple machine learning model in SAP HANA using the python language to predict which customers are likely to churn.
- SAP HANA provides a large set of machine learning libraries to deal with regression, classifications, forecasting etc. problems  
 - The libray we are going to use is the SAP HANA Predictive Analysis Library (PAL)
 - The algorithms in PAL can be accessed through the python language, the documentation of the HANA Python API for PAL can be found here https://help.sap.com/doc/0172e3957b5946da85d3fde85ee8f33d/2.0.03/en-US/html/hana_ml.html# 
 - Here we are using jupyter-notebook as the user interface for the Python scripts and visualizations
 
- Execute (run) the code blocks in this Notebook step by step
- In Exercise 2 and following, the artefacts and SQL code generated here from Python will be build into SAP HANA design-time content and templates constructing a "HANA ML scenario application".

### Step 1 - Load the required packages

In [ ]:
#Import HANA database client library for Python
from hdbcli import dbapi
from hdbcli import dbapi
print(dbapi.__name__)

#Import HANA Dataframe and Algorithm classes from Python API package
from hana_ml import dataframe
from hana_ml.dataframe import ConnectionContext

from hana_ml import algorithms
from hana_ml.algorithms.pal import trees

#Import additional packages
import os
import sys
import configparser
import numpy as np
import matplotlib.pyplot as plt
import logging
from IPython.core.display import Image, display

### Step 2 - Activate logging to access the generated SQL statements

In [ ]:
logger = logging.getLogger()
handler = logging.FileHandler('DAT312_HANAML_SQLtrace.log')
formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)
logger.info("Start logging")

### Step 3 - Create a SAP HANA connection (context)

In [ ]:
#dfctxt = dataframe.ConnectionContext( address="<machine>", port=<SQL-port>,  user="<HANA-user>", password="<password>")
dfctxt = dataframe.ConnectionContext( address="cloudl000022.wdf.sap.corp", 
    port=31053, 
    user="MLLAB_###", 
    password="Welcome19",
    encrypt='true'
    , sslValidateCertificate='false')

### Step 4 - Create SAP HANA dataframe and inspect dataframe attributes

In [ ]:
# Create a dataframe using a SQL Statement against the insurance training data set
train_df =dfctxt.sql ('select * from MLLAB_SHARE.CUSTOMERCHURN_TRAINSAMPLE')

In [ ]:
# Note, unless you use the collect()-method with the dataframe, no data will be transfered from HANA to Python

# review number of records
print('Number of records', train_df.count())

# review number of columns
print('Number of columns', len(train_df.columns))

# review columns names and corresponding SQL types 
print(train_df.dtypes())

### Step 5 - Explore the dataset

In [ ]:
# Explore the target variable 'ContractActivityLABEL' and its value distribution

# customer which cancelled the contract in the past
print('Customer who churned: ', train_df.filter('"ContractActivity" = 1').count()  )
# customer which kept their contract
print('Customer who could be retained: ',train_df.filter('"ContractActivity" = 0').count()  )

# calulate ratio: 
ratio = train_df.filter('"ContractActivity" = 1').count() / train_df.filter('"ContractActivity" = 0').count()
print(ratio)

# If you get ~0.25 as a ration of customers who cancelled their contract, 
# this implies from the training data about 25% of the customers have been cancelling their contract.
# Thus the dataset can be described as slightly imbalanced  between the classed 1 (25%) and 0 (75%).

In [ ]:
#Now let's look at the acual data. Collect() will transfer data from the HANA to the python client)
train_df.head(5).collect()


In [ ]:
# Show column descriptive statistics using the describe method
train_df.describe().collect()

### Step 6 - Fit a Random Forest Classifier model

In [ ]:
#repare feature list 
featurelist = train_df.columns
#print(len(featurelist))
featurelist.remove('AccountID')
featurelist.remove('ContractActivity')
print(len(featurelist))

In [ ]:
# insert a row into the SQLtrace.log file, to better indentify the start of the actual RDT training call
logger.info("-----------------training procedure START ----------------------")

# instantiate the object
rfc = trees.RandomForestClassifier(conn_context=dfctxt, thread_ratio=1.0, n_estimators=200, max_features=5, 
                                   random_state=1234,  min_samples_leaf=1
                                  , max_depth=20
                                   , allow_missing_dependent=False
                                   , split_threshold=0.0000001
                                  ,  calculate_oob=True
                                  , sample_fraction=1
                                , strata=[( 0.5, '0',), ( 0.5, '1')]
                                , priors=[(0.75, '0'), (0.25, '1')]
                                  )    

# perform the training
rfc.fit(train_df, features= featurelist, label = "ContractActivity", key="AccountID")

logger.info("----------------training procedure END ----------------------")


In [ ]:
# Training data confusion matrix
rfc.confusion_matrix_.collect()

In [ ]:
# Calculate Model Accuracy on the given validation dataset
test_df=dfctxt.sql ('select * from MLLAB_SHARE.CUSTOMERCHURN_TESTSAMPLE')

mean_accuracy=rfc.score(test_df, key='AccountID')
print("Accuracy score is {}".format(mean_accuracy))

## Step 7 - Predict with trained model

In [ ]:
# create a dataframe from the test dataset
test_df = dfctxt.sql ('select * from MLLAB_SHARE.CUSTOMERCHURN_TESTSAMPLE')

# testdata set contains 638 records
print(test_df.count())

# The testdata set contains 136 customers who "churned" out of 683.
print('Customer who churned: ',test_df.filter('"ContractActivity" = 1').count())

In [ ]:
# remove the target column and Predict
test_df_trunc = test_df
test_df_trunc.drop(['ContractActivity'])

logger.info("----------------predict START ----------------------")
result_test_pred = rfc.predict(test_df_trunc, key='AccountID', features=featurelist, verbose=False)
logger.info("----------------predict END ----------------------")

In [ ]:
# Prediction result show predicted classification and its confidence value
print(result_test_pred.head(2).collect())

# Overall 86 customers where predicted as likely to churn
print('Overall number of customers predicted to churn: ',result_test_pred.filter('"SCORE" = 1').count())


### Step 8 - Review the SQL trace information

- Open the file DAT312_HANAML_SQLtrace.log
- review sql code between
- -----------------training procedure START ----------------------
-     and
- ----------------training procedure END ----------------------
- inspect the complete SQL code generated by the FIT python method.

###   
### Next exercise 
- For the next exercise you will work in the SAP WebIDE for SAP HANA
- Open webIDE from https://cloudl000022.wdf.sap.corp:53075



###    
###    
###    
### Furher optional steps -- do not continue --  

In [ ]:
#Why are customers interested in buying the new insurance product
rfc.feature_importances_.sort("IMPORTANCE", desc=True).head(10).collect()

### Metrics

In [ ]:
# calculate additional model quality metrics
from hana_ml.algorithms.pal.metrics import accuracy_score 

result_test_pred2=result_test_pred.rename_columns({'AccountID': 'ID'})
test_df2=test_df.rename_columns({'AccountID': 'ID'})

# prepare data for calcualting AUC
auc_in_pred=result_test_pred2.select('ID', 'SCORE',  
                                    ('CASE WHEN SCORE = 1 THEN CONFIDENCE ELSE 1- CONFIDENCE END','PROBABILITY'))

auc_in_observed=test_df2.select(('ID', 'ID_O'),('\"ContractActivity\"', 'ORIGINAL_LABEL'))

auc_in=auc_in_observed.join(auc_in_pred, 'ID_O=ID').select('ID', 'ORIGINAL_LABEL', 'PROBABILITY')
print(auc_in.head(2).collect())

# Calculate AUC
auc, roc = algorithms.pal.metrics.auc(dfctxt, auc_in, positive_label='1')
print('Test data AUC is: ', auc)

In [ ]:
# Plot ROC data
import matplotlib.pyplot as plt
roc_print = roc.collect()
x = roc_print["FPR"]
y = roc_print["TPR"]

plt.plot(x,y)

In [ ]:
#CF-Matrix
CF_IN=auc_in_observed.join(auc_in_pred, 'ID_O=ID').select('ID', 'ORIGINAL_LABEL', ('CAST( SCORE as VARCHAR(1))', 'PREDICTED_LABEL' ))
print(CF_IN.head(5).collect())
#print(CF_IN.dtypes())


cm, cr = algorithms.pal.metrics.confusion_matrix(dfctxt, CF_IN, 'ID','ORIGINAL_LABEL','PREDICTED_LABEL', beta=1 )
#confusion_matrix(conn_context, data, key, label_true=None, label_pred=None, beta=None, native=True)¶
print(cm.collect())
print(cr.collect())

score = algorithms.pal.metrics.accuracy_score( dfctxt, CF_IN, 'ORIGINAL_LABEL', 'PREDICTED_LABEL')
print('Accuracy score is: ', score)